In [1]:
import csv
from xml.dom import minidom

# Parse mast xml output

In [2]:
mast = minidom.parse('mast.xml')
seq_lens = {sequence.attributes['name'].value : sequence.attributes['length'].value for sequence in mast.getElementsByTagName('sequence')}
motif_lens = {motif.attributes['alt'].value.split('-')[-1] : int(motif.attributes['length'].value) for motif in mast.getElementsByTagName('motif')}

In [3]:
mast_motifs_arch = {}

for sequence in mast.getElementsByTagName('sequence'):
    name = sequence.attributes['name'].value#.replace('vdd', 'vanderdonk')
    mast_motifs_arch[name] = []
    for hit in sequence.getElementsByTagName('hit'):
        motif = str(int(hit.attributes['idx'].value) + 1)
        coord = int(hit.attributes['pos'].value) - 1
        end = coord + motif_lens[motif]
        mast_motifs_arch[name].append([motif, coord, end])

# Make annotation

In [4]:
with open('manual_pepts.tsv', 'r') as inf:
    prots = [row[1] for row in csv.reader(inf, delimiter = '\t')]

In [5]:
motifs_for_tree = {}
uniq = {}
for x in mast_motifs_arch:
    lankc = '_'.join(x.split('_')[0:2])
    if x.replace('manual_', '') in prots:
        if lankc not in motifs_for_tree:
            motifs_for_tree[lankc] = []
            uniq[lankc] = []
        
        if [y[0] for y in mast_motifs_arch[x][:-1]] not in uniq[lankc]:
            uniq[lankc].append([y[0] for y in mast_motifs_arch[x][:-1]])
            motifs_for_tree[lankc].append(mast_motifs_arch[x] + [x])

# Write annotation file for iTOL

In [6]:
colors = ["#ffbaff", "#35618f", "#cfe2f2", "#c6dfb5", "#82a0ca"]

In [7]:
n = max([len(x) for x in motifs_for_tree.values()])
n

2

In [8]:
for i in range(1,n + 1):
    with open(f'motif_architectures_{i}.tsv', 'w') as outf:
        writer = csv.writer(outf, delimiter = '\t')
        for x in motifs_for_tree:
            if len(motifs_for_tree[x]) > i-1:
                row = [x, x.replace('_', ' '), seq_lens[motifs_for_tree[x][i-1][-1]]]
                for y in motifs_for_tree[x][i-1][:-1]:
                    row.append('|'.join(['RE', str(y[1]), str(y[2]), colors[int(y[0])-1], y[0]]))
                writer.writerow(row)